<a href="https://colab.research.google.com/github/Mario-Henrique/AssistenteParaCegos/blob/main/AssistenteParaCegos_V01_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Realiza a instalação do pacote de ia do Google

In [ ]:
!pip install -q -U google-generativeai

Configurar a API_KEY do Gemini

In [1]:
import google.generativeai as genai
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)


Para saber quais os modelos disponíveis para uso então vamos listar todos.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Determinar parâmetros de configuração do nosso modelo e
Inicializar o modelo que será utilizado.

In [2]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.25
}

model_text = genai.GenerativeModel(model_name="gemini-pro",
                              generation_config=generation_config)

In [3]:
# Função responsável por receber um questionamento do usuário e retornar
#a resposta do Assistente
def GetAIResponse(question):
  response = model_text.generate_content(question)

  return response.text

Instalação do gTTS para sintetização do texto em voz e do
pysoundfile para execução do audio

In [ ]:
!pip install gTTS
!pip install pysoundfile

Importando a biblioteca IPython e gtts e criar a função responsável por falar um texto recebido.

In [22]:
import IPython
from gtts import gTTS

def ReadText(text):
  #Cria um objeto gtts
  gtts_object = gTTS(text, lang='pt')

  #Indica um caminho para salvar o arquivo gerado e salva o arquivo.
  response_audio = "/content/response_audio.mp3"
  gtts_object.save(response_audio)

  #Toca o arquivo de leitura do texto
  return IPython.display.Audio(response_audio, autoplay=True)

Função utilizada para tirar uma foto e descrever a cena.

In [40]:
# Para efeito de exemplo aqui vou selecionar uma foto aleatoriamente porém a ideia seria realmente
# tirar uma foto usando a câmera do gadget.
import requests
from bs4 import BeautifulSoup
import io
from PIL import Image
from pathlib import Path
import hashlib
from random import randint

# Nesse caso temos um parâmetro que indica que tipo de imagem será buscada.
# Mas na função real não será necessário pois vai apenas tirar a foto e enviar ao Gemini
def TakePhoto():
  # Cria um array de exemplos onde a pessoa poderia estar
  temas = ['Fachada de loja', 'praia movimentada', 'faixa de pedrestes', 'cesta de frutas', 'mesa com vários objetos']
  palavra_chave = temas[randint(0, len(temas)-1)]

  # Envia a requisição para o Google Images
  response = requests.get(f"https://www.google.com/search?q={palavra_chave}&tbm=isch")

  # Verifica se a requisição foi bem-sucedida
  if response.status_code == 200:
      # Extrai os URLs das imagens da página HTML
      soup = BeautifulSoup(response.content, "html.parser")
      imagens = soup.find_all("img", src=True)

      index = randint(1, 5)
      counter = 0
      # Baixa e salva uma imagem aleatória
      for imagem in imagens:
          if counter < index:
              counter = counter + 1
              continue

          url_imagem = imagem["src"]
          if url_imagem.startswith("data:image"):
              continue  # Ignora imagens em base64

          response_imagem = requests.get(url_imagem, stream=True)
          if response_imagem.status_code == 200:
              Image.open(io.BytesIO(response_imagem.content)).convert('RGB').save("pic.jpg")
              break

      # Criação da configuração do modelo
      generation_config = {
          "candidate_count": 1,
          "temperature": 0.25
      }
      # Uma vez tendo a imagem vamos criar o modelo que aceita esse tipo de arquivo
      model = genai.GenerativeModel(model_name="gemini-pro-vision",
                              generation_config=generation_config)

      # Array com as imagens que podemos subir para o modelo
      uploaded_files = []

      # Função responsável por carregar as imagens no array
      def upload_if_needed(pathname: str) -> list[str]:
        path = Path(pathname)
        hash_id = hashlib.sha256(path.read_bytes()).hexdigest()

        try:
          existing_file = genai.get_file(name=hash_id)
          return [existing_file]
        except:
          pass
        uploaded_files.append(genai.upload_file(path=path, display_name=hash_id))
        return [uploaded_files[-1]]

      prompt_model = [
          "Faça uma breve descrição dos elementos presentes na foto.",
          *upload_if_needed("pic.jpg"),
      ]

      response = model.generate_content(prompt_model)

      # Deleta a imagem
      for uploaded_file in uploaded_files:
        genai.delete_file(name=uploaded_file.name)

      # Retorna a descrição da foto em caso de sucesso
      return response.text
  else:
      # Imprime uma mensagem de erro
      print(f"Erro ao buscar imagens: {response.status_code}")
      return False

Conjunto de funções de utilidade que vamos usar para refinar e ajudar em nosso assistente

In [39]:
# Função para dizer se uma palavra específica está no texto
def contem_palavra(frase, palavra):
  # Converter a frase e a palavra para minúsculas
  frase_minuscula = frase.lower()
  palavra_minuscula = palavra.lower()

  # Verificar se a palavra está presente na frase
  if palavra_minuscula in frase_minuscula:
    return True
  else:
    return False

# Função que retorna True caso uma frase seja semelhante a outra em termos semânticos
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def comparar_textos(texto1, texto2):
  # Criar um vetorizador TF-IDF
  vectorizer = TfidfVectorizer()

  # Transformar os textos em vetores de características TF-IDF
  texto1_vetor = vectorizer.fit_transform([texto1])
  texto2_vetor = vectorizer.transform([texto2])

  # Calcular a similaridade cosine entre os vetores
  similaridade = cosine_similarity(texto1_vetor, texto2_vetor)[0][0]

  # Definir um limiar de similaridade
  limiar = 0.65  # Valor entre 0 e 1, quanto maior, mais rigorosa a comparação

  # Retornar True se a similaridade for maior que o limiar, False caso contrário
  return similaridade >= limiar

Instalação do SpeechRecognition para transcrição de aúdio.

In [ ]:
!pip install SpeechRecognition

Capturar aúdio do microfone, verificar se é um comando e realizar a ação necessária!

In [ ]:
import speech_recognition as sr

# Definir a taxa de amostragem e o número de canais de áudio
CHUNK = 1024
RATE = 44100

# Inicializar o recognizer do Google Speech Recognition
recognizer = sr.Recognizer()

# Configurar o microfone como fonte de entrada
mic = sr.Microphone()

with mic as source:
    # Ajustar o nível de ruído do ambiente (opcional)
    recognizer.adjust_for_ambient_noise(source)

    # Informar ao usuário que a gravação está iniciando
    print("Aguardando voz...")

    # Gravar o áudio do microfone
    audio = recognizer.listen(source, chunk_length=CHUNK)

try:
    # Reconhecer a fala no áudio gravado
    texto = recognizer.recognize_google(audio, language='pt-BR')

    if contem_palavra(texto, "Assistente"):
      if comparar_textos("O que você está vendo?", texto):
        ReadText(TakePhoto())
      else:
        ReadText(GetAIResponse(texto))

except sr.UnknownValueError:
    print("Não entendi o que você disse.")
except sr.RequestError as e:
    print("Ocorreu um erro: {0}".format(e))

In [ ]:
#Testando geração de texto com o Gemini e usando recurso de fala.
response = GetAIResponse('De forma direta me diga o que é criatividade?')
ReadText(response)

In [ ]:
# Testando recurso de descrever o que a pessoa "poderia estar vendo".
response = TakePhoto()
ReadText(response)